In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections;
import re;
from time import time

In [2]:
client = Socrata("data.sfgov.org", None)
# results = client.get("cuks-n6tp", limit = 2191368)
data = client.get("cuks-n6tp", limit = 3000000)

In [3]:
data_df = pd.DataFrame.from_records(data)

In [4]:
print(type(data_df))
print(data_df.shape)

<class 'pandas.core.frame.DataFrame'>
(2195020, 13)


In [5]:
print(data_df[:2])

                   address        category                     date dayofweek  \
0    KISSLING ST / 11TH ST  SUSPICIOUS OCC  2015-06-06T00:00:00.000  Saturday   
1  JACKSON ST / WEBSTER ST    NON-CRIMINAL  2015-09-10T00:00:00.000  Thursday   

                       descript incidntnum  \
0       INVESTIGATIVE DETENTION  150494743   
1  AIDED CASE, MENTAL DISTURBED  150795745   

                                            location pddistrict  \
0  {u'type': u'Point', u'coordinates': [-122.4150...   SOUTHERN   
1  {u'type': u'Point', u'coordinates': [-122.4329...   NORTHERN   

             pdid resolution   time                  x                 y  
0  15049474364085       NONE  15:14  -122.415002640244   37.772649875548  
1  15079574564020       NONE  16:40  -122.432920179182  37.7926791576412  


In [6]:
# data_df.sort_values(by=['date', 'time'])
# data_df.to_csv('./data/crimedata.csv', sep= ",", encoding="utf8")

In [7]:
print(data_df.shape)

(2195020, 13)


In [8]:
for col in data_df.columns:
    print(col)

address
category
date
dayofweek
descript
incidntnum
location
pddistrict
pdid
resolution
time
x
y


In [9]:
def convert_date_to_day(dt):
   result = re.findall(r'\d{4}-(\d{2})-(\d{2})T00:00:00.000',dt)
   return result[0][1]
   
def convert_date_to_month(dt):
   result = re.findall(r'\d{4}-(\d{2})-(\d{2})T00:00:00.000',dt)
   return result[0][0]

def convert_time_to_hour(tm):
   result = re.findall(r'(\d{2}):\d{2}',tm)
   return result[0]

In [10]:
data_df = data_df.dropna(how='any',axis=0)
print(data_df[:1])

                 address        category                     date dayofweek  \
0  KISSLING ST / 11TH ST  SUSPICIOUS OCC  2015-06-06T00:00:00.000  Saturday   

                  descript incidntnum  \
0  INVESTIGATIVE DETENTION  150494743   

                                            location pddistrict  \
0  {u'type': u'Point', u'coordinates': [-122.4150...   SOUTHERN   

             pdid resolution   time                  x                y  
0  15049474364085       NONE  15:14  -122.415002640244  37.772649875548  


In [11]:
data_df['day'] = data_df.date.apply(lambda x: convert_date_to_day(x))
data_df['month'] = data_df.date.apply(lambda x: convert_date_to_month(x))
data_df['hour'] = data_df.time.apply(lambda x: convert_time_to_hour(x))
data_df = data_df.sort_values(by=['date','time'])


In [12]:
# df = data_df.drop(['incidntnum','pdid','resolution','x','y', 'date', 'time', 'descript'], axis =1)
df = data_df.drop(['incidntnum','pdid','resolution','date', 'time', 'descript', 'location'], axis =1)
# df = df.drop(df.columns[[0]], axis =1)

for col in df.columns:
    print(col)

address
category
dayofweek
pddistrict
x
y
day
month
hour


In [13]:
# df.sort_values(by=['date'])
df['category'] = df.category.apply(lambda x: x.lower())
df['dayofweek'] = df.dayofweek.apply(lambda x: x.lower())
df['address'] = df.address.apply(lambda x: x.lower())
df['pddistrict'] = df.pddistrict.apply(lambda x: x.lower())

In [14]:
print(df.shape)

(2195019, 9)


In [15]:
series = df.groupby('pddistrict').apply(list)
dictionary = {}


In [16]:
city_target = df.get('category').tolist()
unique_target=set(city_target)
print(len(unique_target))
print(len(city_target))

39
2195019


In [17]:
#For city level
# df = df.drop(['pddistrict', 'category'], axis =1)

In [18]:
for s in series.keys():
    print(str(s))
    mask = df['pddistrict'] == str(s)
#     print(df[mask])
    dictionary[str(s)] = df[mask]

bayview
central
ingleside
mission
northern
park
richmond
southern
taraval
tenderloin


In [19]:
print(type(dictionary['central']))
print(dictionary.keys())

<class 'pandas.core.frame.DataFrame'>
['central', 'northern', 'ingleside', 'park', 'mission', 'tenderloin', 'richmond', 'taraval', 'bayview', 'southern']


In [20]:
def fetchDistrictData(district):
    target = (dictionary.get(district))['category']
#     tempdf = (dictionary.get(district)).drop(['pddistrict', 'category'], axis =1).sort_values(by=['date'])
    tempdf = (dictionary.get(district)).drop(['pddistrict', 'category'], axis =1)
    return tempdf, target
#     dictionary

In [21]:
district_df, district_target = fetchDistrictData('bayview')
print(len(district_target))
print(district_df.shape)

219306
(219306, 7)


In [30]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
enc = OneHotEncoder()

In [32]:
enc.fit(district_df.values.tolist())

ValueError: invalid literal for float(): 1300 block of silver av

In [22]:
from sklearn.model_selection import train_test_split


In [23]:
def fetchTrainTestData(district_df, district_target):
    X_train, X_test, y_train, y_test = train_test_split(district_df, district_target, test_size=0.10, random_state=42)
    return X_train, X_test, y_train, y_test


In [24]:
X_train, X_test, y_train, y_test = fetchTrainTestData(district_df, district_target)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(197375, 7)
(21931, 7)
(197375,)
(21931,)


In [26]:
X_train_list = X_train.values.tolist()
X_test_list = X_test.values.tolist()

In [28]:
print(X_train_list[:1])

[[u'0 block of navy rd', u'monday', u'-122.379424889828', u'37.7294990859712', u'14', u'04', u'19']]


In [28]:
def classify(clf, X_train, Y_train, X_test):
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, Y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)
    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    print(type(pred))
    return pred

In [29]:
from sklearn.preprocessing import StandardScaler
X_train_std = StandardScaler().fit_transform(X_train.values.tolist())
X_test_std = StandardScaler().fit_transform(X_train.values.tolist())

ValueError: invalid literal for float(): 0 block of navy rd

In [63]:
from sklearn.ensemble import AdaBoostClassifier

In [64]:
pred = classify(AdaBoostClassifier(), X_train, X_test, y_train)

Training: 
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)


TypeError: float() argument must be a string or a number